# Zähme die Tabellen des Grauens

Lächle und sei froh, es könnte noch schlimmer kommen. Und ich lächelte und war froh ... und es kam schlimmer.

Die Destatis-Flat-Tables sind im Vergleich zu anderen Datenportalen schon ganz gut.
Das geht geht auch schlimmer: 

Wir zeigen dir jetzt einmal das Datenportal [regionalstatistik.de](https://www.regionalstatistik.de/genesis/online/)

Ziel ist eine Choroplethen-Karte wie diese:

In [1]:
# Display the associated webpage in a new window
import IPython
IPython.display.IFrame('https://datawrapper.dwcdn.net/IJwst/4/', width='600', height='750')

### 1. Daten als .csv herunterladen

Suche auf regionalstatistik.de nach ‘Bauland’ oder direkt nach der Nummer der Tabelle: 61511-01-03-4. Lade die Daten als .csv herunter und speichere sie im ./data Ordner ab.

### 2. Tabelle anschauen

Der erste Fehler fällt dir schon auf, wenn du versuchst, die Tabelle im Jupyter Notebook anzuklicken. 

Du bekommst folgende Fehlermeldung:

```"File Load Error for 61511-01-03-4.csv
/home/p3nny/pfv/data/61511-01-03-4.csv is not UTF-8 encoded"
```

Das heißt, der Zeichensatz, mit dem das Dokument angelegt ist, entspricht nicht dem Standard utf-8 sondern es wird ein deutsches Sonderformat verwendet.

Öffne die .csv bitte mal in Excel oder Google Spreadsheets etc.

Vor Dir siehst du eine recht verschachtelte Tabelle. Sie ist gut geeignet, um auf DIN-A4-Seiten ausgedruckt zu werden, Pandas oder ein Visualisierungstool wie Datawrapper ist damit jedoch überfordert.

### 3. In pandas einlesen

Versuche mal die Tabelle per read_csv in Pandas einzulesen. Denke daran, dass du pandas erst einmal importieren musst.

In [ ]:
import pandas as pd 

In [ ]:
df = pd.read_csv("./data/61511-01-03-4.csv")




Das wird nicht einfach, aber es geht. Allerdings musst du pandas an vielen Stellen auf die Sprünge helfen.

In [ ]:
df = pd.read_csv('./data/61511-01-03-4.csv', sep=';', encoding='ISO-8859-1', skiprows=7, 
                 skipfooter=4, engine="python",
                 decimal=',',
                names=['Jahr', 'AGS', 'Ort', 'Art', 'Einheit', 'Insgesamt', 'baureifes Land'],
                dtype={'AGS': 'str'})

Hier kommt die Schritt für Schritt Erklärung:
    
sep=';' 
In Deutschland wird anders als in den USA nicht das Komma als Trenner verwendet (das ist ja bereits belegt, weil wir '3,3' statt '3.3' schreiben), sondern der Trenner ist ein Semikolon ';'.

encoding='ISO-8859-1'
Setzt das zeichenformat auf 'ISO-8859-1', eine andere oft von deutschen Behörden genutzte Variante ist 'latin-1'. Welches verwendet wurde, weiß man vorher nicht. Deshalb müsst ihr die durchprobieren, falls utf-8 nicht geht. 

skiprows=7
Überspringt den verschachtelten Tabellenkopf. Wie viele Zeilen du überspringen musst, ist von .csv zu .csv unterschiedlich. Auch hier hilft nur ausprobieren.

skipfooter=4, engine="python"
Skipfooter lässt Zeilen unten weg, die nicht zur Tabelle gehören. Diese Funktion erfordert die Angabe engine="python", sonst gibt es eine Fehlermeldung.

decimal=','
Gibt an, dass Python ',' als '.' bei Kommazahlen interpretieren soll: '3.2' statt '3.3'

Damit lässt sich die Tabelle bereits einlesen. Hilfreich sind diese noch diese beiden Parameter:

names=['Jahr', 'AGS', 'Ort', 'Art', 'Einheit', 'Insgesamt', 'baureifes Land']
Gibt an, wie die Spalten heißen sollen

dtype={'AGS': 'str'}
Gibt schon beim einlesen an, dass der AGS als String eingelesen werden soll. Wenn der als Zahl erkannt wird, dann geht die führende Null verloren.


In [ ]:
df

### 3. Entscheiden, was visualisiert werden soll

Um eine Flächenwert-Stufen-Karte (Choroplethenkarte) zu erstellen, musst du zwei Entscheidungen treffen:

- Welche regionale Ebene möchtest du darstellen? In diesem Fall möchten wir Kreise in Nordrhein-Westfalen zeigen.
- Nach welchem Merkmal soll die Karte eingefärbt werden? Um Vergleichbarkeit herzustellen, möchten wir die Kreise anhand des durchschnittlichen Kaufwert je qm einfärben.

### 4. Filtern auf 'durchschnittlicher Kaufwert je qm' 

In [ ]:
df = df[df['Art'] == 'Durchschnittlicher Kaufwert je qm']

In [ ]:
df

### 5. Filtern nach dem Allgemeinen Gemeindeschlüssel, AGS

Der amtliche Gemeindeschlüssel (AGS) ist eigentlich eine achtstellige Zahl, die alle Städte und politisch eigenständigen Gemeinden in Deutschland eindeutig identifiziert.
In unserer Tabelle haben wir Daten bis runter auf Kreisebene, deshalb sind die ersten fünf Stellen angegeben.
Wir interessieren uns nur für NRW, dessen Gemeindeschlüssel beginnen immer mit 05.

[Hier wird der AGS interaktiv erklärt](https://datengui.de/statistik-erklaert/ags)

In [ ]:
df = df[df.AGS.str.startswith('05')]

In [ ]:
df

Leider haben wir in der Tabelle gemischte Werte für das gesamte Bundesland (AGS: 05), für die Regierungsbezirke (AGS zum Beispiel: 051) und für die Kreise (AGS zum Beispiel: 05111).
So können wir die Daten nicht in Datawrapper packen, der erwartet auf einer Kreis-Karte nur Landkreise. 
Deshalb brauchen wir einen Trick, um alle Zeilen rauszufiltern, bei denen der AGS weniger als 5 Stellen hat:

Ein Weg ist für jeden Wert in der Spalte AGS zu gucken, ob die Länge des Strings größer als 3 ist. Erinnert ihr euch an die len('penny) Funktion?

In [ ]:
def check_if_larger_than_3(x):
    if len(x) > 3:
        return True
    else:
        return False

Diese Funktion müssen wir dann auf alle Werte in der Spalte AGS anwenden. Dort wird dann entweder 'False' oder 'True' ausgegeben.

In [ ]:
df.AGS.apply(check_if_larger_than_3)

Pandas kann eine solche Funktion nutzen, um den Dataframe zu filtern: 

In [ ]:
df[df.AGS.apply(check_if_larger_than_3)].head()

Um Schreibarbeit zu sparen, nutzt man in der Datenanalyse einen kürzeren Weg, um die Funktion zu definiieren. Dann sieht das Ganze so aus:

In [ ]:
df = df[df['AGS'].apply(lambda x: len(x)>3)]
df

Das sieht schon ganz gut aus.
Aber ist dir Aachen aufgefallen?

In [ ]:
df[df.Ort.str.contains('Aachen')]

Aus Gründen wird Aachen in der Regionalstatistik oft mehrfach angegeben.
Wir filtern die kreisfreie Stadt und den Kreis Aachen raus.

In [ ]:
df = df[(df.AGS != '05354') & (df.AGS != '05334002') ]

In [ ]:
df

## 6. Spalten auswählen

Wir empfehlen, nur die wirklich notwendigen Spalten in Datwrapper zu kopieren. Deshalb nicht nötige Spalten weglassen:

In [ ]:
df.columns

In [ ]:
df = df[['AGS', 'Ort', 'baureifes Land']]

In [ ]:
df.head()

## 7. Daten in die Zwischenablage kopieren

Statt die Daten als .csv abzuspeichern, kannst du sie auch in die Zwischenablage kopieren (wie mit STRG-C) und in Datawrapper per STRG-V einfügen. 

In [ ]:
df.to_clipboard()

### 8. Baue deine Karte
- In Datawrapper wählst du NRW Landkreise als neue Karte aus
- fügst dann die Daten ein und wählst die entsprechenden Spalten aus (AGS und Wert). AGS steht für 'amtlicher Gemeindeschlüssel'. Datawrapper nutzt diese Gemeindekennziffer, um Daten und Gebiete eindeutig zuzuordnen.
- Danach kannst du die Karte beschriften und deine Datenquelle angeben und die Karte mit Tooltips, Legende etc. nach deinen Wünschen gestalten.
- Am Schluss auf 'Veröffentlichen' klicken.

Prima!

Auf zum nächsten Datenformat:
    
**[>> Json-Notebook](05_json.ipynb)**